### 모듈

In [1]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### 데이터 불러오기

In [2]:
filenames = [
    "labeled_bond.csv",
    "labeled_economy.csv",
    "labeled_edaily.csv",
    "labeled_hk.csv",
    "labeled_infomax.csv",
    "labeled_mpb.csv"
]

df_list = []
for file in filenames:
    temp_df = pd.read_csv(file)
    df_list.append(temp_df)

# 모든 데이터가 합쳐진 최종 데이터프레임
df = pd.concat(df_list, ignore_index=True)

print(df.head())
print(df.shape)

         date                                        ngram_label
0  2024-11-29  [(('국내',), 22, -1), (('위험',), 25, -1), (('추가',...
1  2024-11-28  [(('의견',), 16, -1), (('압력',), 29, -1), (('투자',...
2  2024-11-27  [(('금리',), 90, None), (('잔고',), 15, None), (('...
3  2024-11-26  [(('글로벌',), 25, None), (('유동성',), 68, None), (...
4  2024-11-25  [(('전략',), 19, 1), (('유로존',), 47, 1), (('인하',)...
(16870, 2)


In [3]:
df['parsed'] = df['ngram_label'].apply(ast.literal_eval)

# 파싱된 열을 사용해서 각 라벨별 개수를 계산
none_count = df['parsed'].apply(lambda lst: sum(1 for item in lst if item[2] is None)).sum()
one_count = df['parsed'].apply(lambda lst: sum(1 for item in lst if item[2] == 1)).sum()
zero_count = df['parsed'].apply(lambda lst: sum(1 for item in lst if item[2] == 0)).sum()
minus_one_count = df['parsed'].apply(lambda lst: sum(1 for item in lst if item[2] == -1)).sum()

print("None 라벨 개수:", none_count)
print("1 라벨 개수:", one_count)
print("0 라벨 개수:", zero_count)
print("-1 라벨 개수:", minus_one_count)
print("총 개수:", none_count + one_count + zero_count + minus_one_count)

None 라벨 개수: 311687
1 라벨 개수: 195491
0 라벨 개수: 39249
-1 라벨 개수: 156804
총 개수: 703231


In [4]:
def parse_ngram_list(x):
    return ast.literal_eval(x)

In [5]:
df['ngram_list'] = df['ngram_label'].apply(parse_ngram_list)

### NBC 모델링

In [6]:
def create_document_and_target(ngram_list):
    if not ngram_list:
        return "", None
    
    # 초기화
    tokens = []
    target = None

    for item in ngram_list:
        token_tuple, freq, lab = item
        # item이 (('금리',), 13, 0) 형태일 때,
        # item[0] = 토큰 튜플
        # item[1] = 빈도수(freq)
        # item[2] = 라벨(-1이면 dovish, 1이면 hawkish)

        # 튜플을 문자열로 만들어 합친다.
        token_str = "_".join(token_tuple)
        if token_str not in tokens:
            tokens.append(token_str)

        if lab == 1:
            target = "hawkish"
        elif lab == -1:
            target = "dovish"

    document = " ".join(tokens) if tokens else ""
    return document, target

In [7]:
df[['document', 'target']] = df['ngram_list'].apply(lambda x: pd.Series(create_document_and_target(x)))
df = df.dropna(subset=['document', 'target'])
df.head()

,date,ngram_label,parsed,ngram_list,document,target
0,2024-11-29,"[(('국내',), 22, -1), (('위험',), 25, -1), (('추가',...","[((국내,), 22, -1), ((위험,), 25, -1), ((추가,), 74,...","[((국내,), 22, -1), ((위험,), 25, -1), ((추가,), 74,...",국내 위험 추가 기대 시장 금리 둔화 하락 변동성 유지 폭 확대 요인 관련 삭제 판...,dovish
1,2024-11-28,"[(('의견',), 16, -1), (('압력',), 29, -1), (('투자',...","[((의견,), 16, -1), ((압력,), 29, -1), ((투자,), 36,...","[((의견,), 16, -1), ((압력,), 29, -1), ((투자,), 36,...",의견 압력 투자 판단 관련 금통위 추가 인하 한국은행 금리 동결 미국 정책 불확실성...,dovish
4,2024-11-25,"[(('전략',), 19, 1), (('유로존',), 47, 1), (('인하',)...","[((전략,), 19, 1), ((유로존,), 47, 1), ((인하,), 43, ...","[((전략,), 19, 1), ((유로존,), 47, 1), ((인하,), 43, ...",전략 유로존 인하 경기 독일 제조업 중심 둔화 우려 수출 경제 대응 중국 주요 트럼...,hawkish
5,2024-11-22,"[(('Fed',), 28, 1), (('fed',), 25, 1), (('전체',...","[((Fed,), 28, 1), ((fed,), 25, 1), ((전체,), 18,...","[((Fed,), 28, 1), ((fed,), 25, 1), ((전체,), 18,...",Fed fed 전체 추이 국채 대비 준비금 금리 대비_대비,hawkish
9,2024-11-18,"[(('채권',), 63, -1), (('주요',), 15, -1), (('국채',...","[((채권,), 63, -1), ((주요,), 15, -1), ((국채,), 56,...","[((채권,), 63, -1), ((주요,), 15, -1), ((국채,), 56,...",채권 주요 국채 금리 재료 부동산 상승 대출 정부 증가 분기 투자 월간 크 둔화 압...,dovish


### 학습

In [8]:
# 피처(X)와 타겟(y) 준비
X = df['document']
y = df['target']

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# CountVectorizer로 문서 벡터화
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 나이브 베이즈 분류기(MultinomialNB) 학습
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)

# 테스트 데이터 예측 및 평가
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      dovish       0.57      0.55      0.56       304
     hawkish       0.62      0.64      0.63       348

    accuracy                           0.60       652
   macro avg       0.60      0.59      0.59       652
weighted avg       0.60      0.60      0.60       652



In [9]:
feature_names = vectorizer.get_feature_names_out()
probs = np.exp(clf.feature_log_prob_)

In [10]:
sentiment_score = probs[1] - probs[0]

sentiment_df = pd.DataFrame({
    'feature': feature_names,
    'dovish_prob': probs[0],
    'hawkish_prob': probs[1],
    'sentiment_score': sentiment_score
})

print("Hawkish 쪽 상위 단어:")
print(sentiment_df.sort_values('sentiment_score', ascending=False).head(10))
print("\nDovish 쪽 상위 단어:")
print(sentiment_df.sort_values('sentiment_score', ascending=True).head(10))

Hawkish 쪽 상위 단어:
      feature  dovish_prob  hawkish_prob  sentiment_score
9889    인플레이션     0.002838      0.004441         0.001603
9806       인상     0.006699      0.008170         0.001471
2851       긴축     0.000921      0.002310         0.001389
4526       물가     0.005785      0.007037         0.001253
2251    금리_인상     0.004518      0.005759         0.001241
13060      침체     0.000732      0.001806         0.001074
311        가격     0.004145      0.005192         0.001047
2939       내년     0.001964      0.002798         0.000834
874     경기_침체     0.000379      0.001206         0.000826
2752       기준     0.004057      0.004834         0.000776

Dovish 쪽 상위 단어:
      feature  dovish_prob  hawkish_prob  sentiment_score
9936       인하     0.005541      0.003858        -0.001683
2152       금리     0.016257      0.014927        -0.001330
2283    금리_인하     0.003461      0.002282        -0.001179
4622       미국     0.014299      0.013362        -0.000937
13574     트럼프     0.001504      0.0008

In [12]:
dovish_idx = np.where(clf.classes_ == 'dovish')[0][0]
hawkish_idx = np.where(clf.classes_ == 'hawkish')[0][0]

dovish_probs = probs[dovish_idx]
hawkish_probs = probs[hawkish_idx]

In [25]:
import numpy as np

N = 10000
# dovish에서 확률이 높은 단어 상위 N개
top_dovish_indices = np.argsort(dovish_probs)[::-1][:N]
top_dovish_words = [(feature_names[i], dovish_probs[i]) for i in top_dovish_indices]

# hawkish에서 확률이 높은 단어 상위 N개
top_hawkish_indices = np.argsort(hawkish_probs)[::-1][:N]
top_hawkish_words = [(feature_names[i], hawkish_probs[i]) for i in top_hawkish_indices]

print("===== Top Dovish Words =====")
for word, prob in top_dovish_words:
    print(word, prob)

print("===== Top Hawkish Words =====")
for word, prob in top_hawkish_words:
    print(word, prob)

===== Top Dovish Words =====
금리 0.016256638127235292
미국 0.014299067952747367
상승 0.012903706513493016
시장 0.011210306708572669
하락 0.010952909938224774
경제 0.010221361222499193
대비 0.00936788772081934
투자 0.008101224666738916
경기 0.007539015931505366
대출 0.007125826379104805
인상 0.006699089628264879
증가 0.006570391243090931
거래 0.006360409667280807
중국 0.0061301072938116374
국채 0.006014956107077057
확대 0.005960767313319604
은행 0.005913352118781835
외국인 0.005798200932047253
물가 0.005784653733607892
분기 0.0055408041616993595
인하 0.0055408041616993595
국내 0.0053714641812073255
채권 0.005222444998374335
기업 0.005107293811639754
금융 0.005059878617101984
계약 0.004958274628806762
우려 0.00488376503739027
기대 0.004849897041291862
지속 0.004727972255337596
국고채 0.004612821068603012
정부 0.004612821068603012
기준금리 0.004592500270943968
강세 0.004558632274845563
금리_인상 0.004517990679527473
발행 0.004450254687330658
추가 0.004443481088110978
포인트 0.004416386691232254
국채선물 0.00440283949279289
지수 0.004328329901376395
규모 0.004260593909179584


In [26]:
import pandas as pd

# 각 리스트에서 단어만 추출 (확률 정보는 제외)
dovish_words = [word for word, prob in top_dovish_words]
hawkish_words = [word for word, prob in top_hawkish_words]

df = pd.DataFrame({
    "dovish": pd.Series(dovish_words),
    "hawkish": pd.Series(hawkish_words)
})

# CSV 파일로 저장 (index 없이 저장)
df.to_csv("word_dictionary.csv", index=False)